<a href="https://colab.research.google.com/github/sebasmos/satellite.extractor/blob/main/downloader_sentinel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
% cd /content/drive/MyDrive/Dengue_prediction

/content/drive/MyDrive/Dengue_prediction


# Setup

In [3]:
!pip install sentinelhub
! pip install sentinelhub --upgrade

     |████████████████████████████████| 201 kB 5.0 MB/s 
     |████████████████████████████████| 3.1 MB 43.7 MB/s 
     |████████████████████████████████| 131 kB 49.6 MB/s 
     |████████████████████████████████| 8.3 MB 19.4 MB/s 
     |████████████████████████████████| 6.3 MB 37.3 MB/s 
     |████████████████████████████████| 128 kB 59.3 MB/s 
     |████████████████████████████████| 79 kB 6.8 MB/s 
     |████████████████████████████████| 127 kB 58.8 MB/s 
     |████████████████████████████████| 47 kB 4.8 MB/s 
  Created wheel for sentinelhub: filename=sentinelhub-3.4.1-py3-none-any.whl size=216207 sha256=ffbd39f0c863d86c5d9ec29df4f7b68b1ccdf9b932b27fa15c0e4f9359886613
  Stored in directory: /root/.cache/pip/wheels/90/10/49/ed51724fd18155810110102bdabd3191ac6676843a4e77198d
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6109 sha256=d859f201cd868ccba1d7b1fc8364c5079cfa71eefdf38c2805f6e2dd4144464d
  Stored in directory: /root/.cache/pip/wheels/a5/b0/12/7ee4fdb0f9fbb415

In [8]:
% cd /content/drive/MyDrive/Dengue_prediction/Codes_download_data_sentinel/
#!git clone https://github.com/sentinel-hub/sentinelhub-py.git

/content/drive/MyDrive/Dengue_prediction/Codes_download_data_sentinel


In [9]:
% cd /content/drive/MyDrive/Dengue_prediction/Codes_download_data_sentinel/sentinelhub-py
# Install dependencies
! python setup.py build
! python setup.py install
! pip install -r requirements.txt

/content/drive/MyDrive/Dengue_prediction/Codes_download_data_sentinel/sentinelhub-py
running build
running build_py
running egg_info
writing sentinelhub.egg-info/PKG-INFO
writing dependency_links to sentinelhub.egg-info/dependency_links.txt
writing entry points to sentinelhub.egg-info/entry_points.txt
writing requirements to sentinelhub.egg-info/requires.txt
writing top-level names to sentinelhub.egg-info/top_level.txt
reading manifest file 'sentinelhub.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE.md'
writing manifest file 'sentinelhub.egg-info/SOURCES.txt'
running install
running bdist_egg
running egg_info
writing sentinelhub.egg-info/PKG-INFO
writing dependency_links to sentinelhub.egg-info/dependency_links.txt
writing entry points to sentinelhub.egg-info/entry_points.txt
writing requirements to sentinelhub.egg-info/requires.txt
writing top-level names to sentinelhub.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
adding li

In [11]:
% cd /content/drive/MyDrive/Dengue_prediction/Codes_download_data_sentinel

/content/drive/MyDrive/Dengue_prediction/Codes_download_data_sentinel


In [12]:
!git clone https://github.com/sebasmos/satellite.extractor.git

Cloning into 'satellite.extractor'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 60 (delta 19), reused 54 (delta 13), pack-reused 0
Unpacking objects: 100% (60/60), done.


# Import functions

In [13]:
import pandas as pd
import glob
import json
import datetime as dt
from urllib.parse import unquote
from bs4 import BeautifulSoup
import os
import shutil
from sentinelhub import SHConfig
import datetime
import numpy as np
import matplotlib.pyplot as plt
import sys
from datetime import timedelta
sys.path.insert(0,'..') 
from sentinelhub import MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient, DataCollection, bbox_to_dimensions, DownloadRequest
!pip install epiweeks
from epiweeks import Week, Year
from datetime import date
import glob, shutil

In [14]:
!ls

data			   satellite_imagery_medellin_2015_2018_colab.ipynb
DATASET_improved_v1	   Sentinel_Data_Medellin
downloader_sentinel.ipynb  sentinelhub-py
satellite.extractor


In [19]:
% cd /content/drive/MyDrive/Dengue_prediction/Codes_download_data_sentinel/satellite.extractor
!ls

/content/drive/MyDrive/Dengue_prediction/Codes_download_data_sentinel/satellite.extractor
coordinates.txt  extractor.py  linux	  requirements.txt  setup.py
Dockerfile	 __init__.py   README.md  sentinelhub


In [46]:
from linux.request_multiple_images_linux import download_multiple_images
from linux.request_multiple_images_linux import get_folder_ID
from linux.rename_linux import get_request_individual
from extractor import get_images

In [21]:
dic = {'Cali': [-76.55323133405957, 3.4062672204498083, -76.4860619546151, 3.473745747318065],
    	    'Medellín': [-75.60847058045997, 6.210967220073269, -75.5410220422612, 6.278445747318065],
    	    'Villavicencio': [-73.66295855834497, 4.108767220355771, -73.59573458823247, 4.176245747318064],
    	    'Cúcuta': [-72.53849203162315, 7.873767219848251, -72.47080007014465, 7.941245747318064],
    	    'Ibagué': [-75.2341715228093, 4.404067220316198, -75.16692155913864, 4.471545747318066]}

In [32]:
CLIENT_ID = "***"
CLIENT_SECRET = "***"
# Define the range of years
img_format = "tiff"
'''
    # REAL DATA FOR COLOMBIA
    initial_year = 2015
    end_year = 2018
    years = list(range(initial_year,end_year))
    first_2015_week = 44
    end_year = 2020
    start_2015 = Week(initial_year,first_2015_week).startdate()
    weeks_2015 = list(range(first_2015_week, 53))
    weeks = list(range(1,53))
    
    # TESTING DATA
    
    initial_year = 2017
    end_year = 2018
    years = list(range(initial_year,end_year))
    first_2015_week = 44
    end_year = 2018
    start_2015 = Week(initial_year,first_2015_week).startdate()
    weeks_2015 = list(range(first_2015_week, 53))
    weeks = list(range(1,10))
'''
    
initial_year = 2015
end_year = 2018
years = list(range(initial_year,end_year))
first_2015_week = 44
end_year = 2020
start_2015 = Week(initial_year,first_2015_week).startdate()
weeks_2015 = list(range(first_2015_week, 53))
weeks = list(range(1,53))
    
root_images = "/data/"
       
temporal_path = os.path.abspath(os.getcwd()) + root_images  
    
if not os.path.isdir(temporal_path):
    os.makedirs(temporal_path)
    print("Creating temporal data folder")
        
print(f"Number of cities: {len(dic)}")
    

Number of cities: 5


In [ ]:
for i in dic: 
    print(f"City: {i} - Coordinates: {dic[i]}")	
      
    current_coor = dic[i]
        
    city_str = "DATASET" + "/" + str(i)
        
    if not os.path.exists(city_str):
        os.makedirs(city_str)
    
    # Download images on given range
    get_images(current_coor, years, weeks, weeks_2015, img_format, root_images, CLIENT_ID, CLIENT_SECRET)
      
    # Move to structured folder in DATASETS
    root_images_store = "." + root_images
    dataset_store = "./" + city_str
    for root, dirs, files in os.walk(root_images_store, topdown=True):
        for name in files:
          path = os.path.join(root, name)
          #print(path)
          if img_format in path and not dataset_store in path:
              shutil.copy(path, dataset_store)
              
    # images = glob.glob(dataset_store+"/*")

City: Cali - Coordinates: [-76.55323133405957, 3.4062672204498083, -76.4860619546151, 3.473745747318065]
Year: 2015 - week: 44
Requested week slot: 11/01/2015 - 11/08/2015 
Ended downloading
Image empty:  0
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
Slot corrected to:  [(datetime.date(2015, 11, 1), datetime.date(2015, 11, 24))]
image_2015-11-01
./data/2015/6a193bfd07c70a5c8345a9085ae2c1a4/response.tiff renamed to ./data/2015/6a193bfd07c70a5c8345a9085ae2c1a4/image_2015-11-01.tiff
Year: 2015 - week: 45
Requested week slot: 11/08/2015 - 11/15/2

/usr/local/lib/python3.7/dist-packages/sentinelhub/download/sentinelhub_client.py:71: SHRateLimitWarning: Download rate limit hit
  warnings.warn('Download rate limit hit', category=SHRateLimitWarning)


This image is being replaced..
This image is being replaced..
Slot corrected to:  [(datetime.date(2016, 2, 21), datetime.date(2016, 3, 23))]
image_2016-02-21
./data/2016/6c20d3554d53ddd4dd258de0205b0a52/response.tiff renamed to ./data/2016/6c20d3554d53ddd4dd258de0205b0a52/image_2016-02-21.tiff
Year: 2016 - week: 9
Requested week slot: 02/28/2016 - 03/06/2016 
Ended downloading
Image empty:  0
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..


/usr/local/lib/python3.7/dist-packages/sentinelhub/download/sentinelhub_client.py:71: SHRateLimitWarning: Download rate limit hit
  warnings.warn('Download rate limit hit', category=SHRateLimitWarning)


This image is being replaced..


...

This image is being replaced..


/usr/local/lib/python3.7/dist-packages/sentinelhub/download/sentinelhub_client.py:71: SHRateLimitWarning: Download rate limit hit
  warnings.warn('Download rate limit hit', category=SHRateLimitWarning)


This image is being replaced..
This image is being replaced..


/usr/local/lib/python3.7/dist-packages/sentinelhub/download/sentinelhub_client.py:71: SHRateLimitWarning: Download rate limit hit
  warnings.warn('Download rate limit hit', category=SHRateLimitWarning)


This image is being replaced..


In [37]:
shutil.rmtree("DATASET")
shutil.rmtree("data")